In [ ]:
# PASTE THIS TO THE FIRST CELL OF THE NOTEBOOK IN ORDER TO HAVE WORKING IMPORTS
import sys
import os
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)

In [ ]:
from sklearn.cluster import OPTICS
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

## data import

In [ ]:
from src.features.get_x_y_tuple_list import get_x_y_tuple_list
from src.features.get_first_and_last_x_y_coordinates import get_first_and_last_x_y_coordinates 
from src.features.get_first_x_y_coordinates import get_first_x_y_coordinates
from src.features.get_last_x_y_coordinates import get_last_x_y_coordinates
def import_data(df_path):
    df_cuid = pd.read_csv(df_path)
    df_cuid_grouped_path = df_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    # 1.1 CONVERT FEATURES TO NUMBERS
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
    list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])
    first_last_x_coords, first_last_y_coords = get_first_and_last_x_y_coordinates(list_x_y_tuples)
    X = np.array([first_last_x_coords, first_last_y_coords]).T

    first_x_coords, first_y_coords = get_first_x_y_coordinates(list_x_y_tuples)
    first_coordinates = np.array([first_x_coords, first_y_coords]).T

    last_x_coords, last_y_coords = get_last_x_y_coordinates(list_x_y_tuples)
    last_coordinates = np.array([last_x_coords, last_y_coords]).T
    return df_cuid, df_cuid_grouped, X, first_coordinates, last_coordinates, list_x_y_tuples
        

In [ ]:
df_path_k729_2022 = f'{parent_parent_dir}/data/processed/k729_2022_cuid.csv'
df_path_k733_2020 = f'{parent_parent_dir}/data/processed/k733_2020_cuid.csv'
df_path_k733_2018 = f'{parent_parent_dir}/data/processed/k733_2018_cuid.csv'

df_cuid_k729_2022, df_cuid_grouped_k729_2022, X_k729_2022, first_coordinates_k729_2022, last_coordinates_k729_2022, list_x_y_tuples_k729_2022 = import_data(df_path_k729_2022)
df_cuid_k733_2020, df_cuid_grouped_k733_2020, X_k733_2020, first_coordinates_k733_2020, last_coordinates_k733_2020, list_x_y_tuples_k733_2020 = import_data(df_path_k733_2020)
df_cuid_k733_2018, df_cuid_grouped_k733_2018, X_k733_2018, first_coordinates_k733_2018, last_coordinates_k733_2018, list_x_y_tuples_k733_2018 = import_data(df_path_k733_2018)

In [ ]:
from src.visualization.plot_vehicle_tracks_in_notebook import plot_vehicle_tracks_in_notebook
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k729_2022, df_cuid_grouped_k729_2022)

indices top right -> top middle

2 4

indices bottom mid -> top left

9 10 70 71 

In [ ]:
indices_group_1 = [9, 10, 70, 71]
indices_group_2 = [2, 4]
# filter dataframe for indices of interest
df_cuid_grouped_k729_2022_group_1 = df_cuid_grouped_k729_2022[df_cuid_grouped_k729_2022['track_id'].isin(indices_group_1)]
df_cuid_grouped_k729_2022_group_2 = df_cuid_grouped_k729_2022[df_cuid_grouped_k729_2022['track_id'].isin(indices_group_2)]

# create x y tuple lists for each group
df_cuid_grouped_k729_2022_group_1 = df_cuid_grouped_k729_2022_group_1.reset_index(drop=True)
df_cuid_grouped_k729_2022_group_2 = df_cuid_grouped_k729_2022_group_2.reset_index(drop=True)
list_x_y_tuples_group_1 = get_x_y_tuple_list(df_cuid_grouped_k729_2022_group_1, ['x','y'])
list_x_y_tuples_group_2 = get_x_y_tuple_list(df_cuid_grouped_k729_2022_group_2, ['x','y'])

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np

def generate_new_paths(paths, num_new_paths):
    # Flatten the paths (each path is a sequence of (x, y) points)
    flattened_data = np.vstack(paths)

    # Fit a Gaussian Mixture Model
    gmm = GaussianMixture(n_components=5, covariance_type='full')
    gmm.fit(flattened_data)

    # Generate new points from the GMM
    new_paths = []
    for _ in range(num_new_paths):
        new_path = gmm.sample(flattened_data.shape[0])[0]  # Generate new samples
        
        # Split the new_path into x and y components
        x_values = new_path[:, 0]  # First column corresponds to x values
        y_values = new_path[:, 1]  # Second column corresponds to y values
        
        # Append the x and y lists as tuples to the new_paths list
        new_paths.append((x_values.tolist(), y_values.tolist()))

    return new_paths


In [ ]:
new_paths_group_1 = generate_new_paths(list_x_y_tuples_group_1, 10)
new_paths_group_2 = generate_new_paths(list_x_y_tuples_group_2, 10)

In [ ]:
# append new paths to df
df_new_paths_group_1 = pd.DataFrame(new_paths_group_1, columns=['x', 'y'])
df_new_paths_group_2 = pd.DataFrame(new_paths_group_2, columns=['x', 'y'])
# append new df to old df
df_additional_paths = pd.concat([df_cuid_grouped_k729_2022, df_new_paths_group_1, df_new_paths_group_2], ignore_index=True)
df_additional_paths = df_additional_paths.reset_index(drop=True)
df_additional_paths['track_id'] = df_additional_paths.index
df_additional_paths = df_additional_paths.loc[:, ~df_additional_paths.columns.str.contains('^Unnamed')]

In [ ]:
# additional_paths_optimization
from src.models.optics.optimize_optics_parallelized_no_aic_bic import optimize_optics_parallelized_no_aic_bic
list_x_y_tuples_additional_paths = get_x_y_tuple_list(df_additional_paths, ['x','y'])
first_last_x_coords_additional_paths, first_last_y_coords_additional_paths = get_first_and_last_x_y_coordinates(list_x_y_tuples_additional_paths)
X_additional_paths = np.array([first_last_x_coords_additional_paths, first_last_y_coords_additional_paths]).T

In [ ]:
import json 
with open(f'{parent_parent_dir}/src/models/optics/optimized_optics_clustering_parameters/optimization_parameters_clustering_start_end_report_notebook.json') as f:
    optimization_parameters = json.load(f)
print(optimization_parameters)

In [ ]:
optimization_results_additional_paths = optimize_optics_parallelized_no_aic_bic(X_additional_paths, 
                                        max_eps_range=optimization_parameters['max_eps_range'], 
                                        min_samples_range=optimization_parameters['min_samples_range'], 
                                        metrics=optimization_parameters['metrics'],
                                        cluster_methods=optimization_parameters['cluster_methods'],
                                        xis=optimization_parameters['xis']
)

In [ ]:
print(optimization_results_additional_paths)

In [ ]:
from src.models.ensure_distance_metric_params import ensure_distance_metric_params
from src.models.DISTANCE_METRICS_WITH_ADDITIONAL_ARGS import DISTANCE_METRICS_WITH_ADDITIONAL_ARGS
def get_optimized_model_from_eval_metric(X, eval_metric, optimized_params):
    if optimized_params[eval_metric]['metric'] in DISTANCE_METRICS_WITH_ADDITIONAL_ARGS: 
        metric_params = ensure_distance_metric_params(X, [optimized_params[eval_metric]['metric']])[optimized_params[eval_metric]['metric']]
    else:
        metric_params = {}
    xi_kwarg = {'xi': optimized_params[eval_metric]['xi']} if optimized_params[eval_metric]['xi'] is not None else {}
    return OPTICS(max_eps=optimized_params[eval_metric]['epsilon'],
                  min_samples=optimized_params[eval_metric]['min_samples'],
                  metric=optimized_params[eval_metric]['metric'],
                  cluster_method=optimized_params[eval_metric]['cluster_method'],
                  metric_params=metric_params,
                  **xi_kwarg)

optics_model_additional_paths = get_optimized_model_from_eval_metric(X_additional_paths, 'silhouette', optimization_results_additional_paths)
optics_model_additional_paths.fit(X_additional_paths)

In [ ]:
from src.models.optics.calculate_cluster_medoids import calculate_cluster_medoids
medoids_additional_paths = calculate_cluster_medoids(X_additional_paths, optics_model_additional_paths)

In [ ]:
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k729_2022, df_additional_paths, 'k729_2022 vehicle tracks and optics medoids for additional paths', color='gray')
plt.gca().scatter(medoids_additional_paths[:,0], medoids_additional_paths[:,1], c='red', s=100)
# plot radius around medoids
for medoid in medoids_additional_paths:
    plt.gca().add_artist(plt.Circle(medoid, optics_model_additional_paths.get_params()['max_eps'], color='red', fill=False))